# Question 3

![](images/question_3.png)

![](images/question_3_diag.png)

![](images/question_3_a.png)

# ASSUMPTIONS: <br>
### - IF A TREE BURNED, WE CAN STILL MOVE ON IT <br>
### - IF THE FIRE GOES OFF THE GRID,  IT IS EXTINGUISHED <br>  
### - THERE ARE MULTIPLE FIRES (SINCE PROBABILITY DOESN'T ADD UP TO 1). THEY SPREAD IF THEIR CHANCE IS UP.

We want our true value, with 95% probability, to be within 0.005 of the true answer. <br>
$P\{|\hat{p} - p| > \epsilon \} \leq \alpha$, where $\epsilon=0.005$, and $\alpha = 0.05$ (since we want 95% accuracy), so: <br>
$P\{|\hat{p} - p| > 0.005\} \leq 0.05$. We do now have an intelligent guess here, so we will calculate: $N = 0.25 (\frac{z_{0.05}}{0.005})^2$ = $0.25 (\frac{1.6449}{0.005})^2 \approx 27057$ iterations. 

Entire question will be posted so that decesions could be explained.
![](images/question_3_parts.png)

In [ ]:
import random
import numpy as np

In [ ]:
trials : int = 27057

# We will track the x and y of the fire. We can keep track of trees being spread to
# by setting a max bound
max_row : int = 20
max_col : int = 50

# The house coordinates are:
house_row : int = 0
house_col : int = 49

# Since question (a) asks for a propostion, we should also calculate total cells
total_cells = max_row * max_col

# We will also track of ACTIVE fire locations (row,col)
active_fires : list = []

# Track porbability of moving a direction
prob_left : float = 0.8
prob_right : float = 0.3
prob_up : float = 0.3
prob_down : float = 0.3

# We will keep track of indicies of trees already burned. 
already_burned : list = []

In [ ]:
# Let's write a function for making moving easier
def possible( pos : tuple ) -> bool: # Check if move is possible (we can't go off border)
    return pos[0] < max_row and pos[0] >= 0 and pos[1] < max_col and pos[1] >= 0

def move( pos : tuple, direction : str ) -> None:
    global active_fires
    if direction == "place": # We don't move but place
        active_fires.append( pos ) # We place the new fire
        if pos not in already_burned: # If didin'talready burn
            already_burned.append(pos)  # Now we say it burned
    if direction == "u" and possible((pos[0]-1, pos[1])): # up
        move( (pos[0] - 1, pos[1]), "place" ) # places
    if direction == "d" and possible((pos[0]+1, pos[1])): # down
        move( (pos[0] + 1, pos[1]), "place" )# places
    if direction == "l" and possible((pos[0], pos[1]-1)): # left
        move( (pos[0] , pos[1] - 1), "place" ) # places
    elif direction == "r" and possible((pos[0], pos[1]+1)): # right
        move( (pos[0], pos[1] + 1), "place" ) # places
        
def process_move(  ) -> None:
    global active_fires
    global prob_down
    global prob_up
    global prob_right
    global prob_left
    # Save copy of activactive_fires because we are resetting activeactive_fires as this iterates
    active_fires_copy = active_fires # Here save
    active_fires = [] # Here we reset
    # Check if we are moving in up, down, left, or right for each active fire
    for fire in active_fires_copy:
        if random.random() < prob_left:
            move(fire, "l")
        if random.random() < prob_up:
            move(fire, "u")
        if random.random() < prob_left:
            move(fire, "l")
        if random.random() < prob_right:
            move(fire, "r")
    # Now we should have our updates fires           



In [ ]:
# We need a reset as well for the trials
def reset() -> None:
    global active_fires
    global already_burned
    already_burned = []
    active_fires = []
    move( (0,0), "place" )

In [ ]:
# Let's track some variables 
proportions_burned = []
trials = 3
# Now let's run this simulation
for trial in range(trials):
    reset()
    while len(active_fires) != 0: # While there are still fires to process
        process_move()
    proportions_burned.append( len(already_burned) / total_cells )
proportions_burned = np.array(proportions_burned)
print("\t{}".format( proportions_burned.sum() / (total_cells * trials) ))